In [ ]:
import sys
sys.path.append("..")

In [ ]:
from emeval.input.spec_details import FileSpecDetails

sd = FileSpecDetails("data/", "shankari@eecs.berkeley.edu", "train_bus_ebike_mtv_ucb")

In [ ]:
from emeval.input.phone_view import PhoneView

pv = PhoneView(sd)

In [ ]:
import pandas as pd

def get_dfs(pv_map):
    """
    Returns a dictionary consisting of versions of the dataframes in a
    PhoneView map with uniquely-identifiable (i.e. fully queryable) rows.
    """
    df_map = dict()
    pkey = set()
    
    def get_pkey(node, curr_pkey):
        for key, val in node.items():
            if isinstance(val, list):
                for e in val:
                    get_pkey(e, curr_pkey)
            elif isinstance(val, dict):
                get_pkey(val, curr_pkey)
            elif isinstance(val, pd.DataFrame):
                for col in val.columns.values:
                    if col in curr_pkey:
                        curr_pkey.remove(col)
                if key not in df_map:
                    df_map[key] = val
            else:
                curr_pkey.add(key)
    
    get_pkey(pv_map, pkey)
    print(f"determined {pkey=}", end="\n\n")
    
    def prepend_pkey(node):
        for key, val in node.items():
            if isinstance(val, list):
                for e in val:
                    prepend_pkey(e)
            elif isinstance(val, dict):
                prepend_pkey(val)
            elif key in pkey:
                for df_label, df in df_map.items():
                    if key not in df.columns.values:
                        df_map[df_label] = (
                            df.assign(
                                **{key: val},
                                ignore_index=True
                            )[[key] + [c for c in df.columns.values if c != key]]
                        )
                        print(f"updated columns of {df_label=}: {df_map[df_label].columns.values}", end="\n\n")
                    
    prepend_pkey(pv_map)
    
    for df in df_map.values():
        assert all(p in df.columns.values for p in pkey)
    
    return df_map

df_map = get_dfs(pv.map())

In [ ]:
from IPython.display import display

for label, df in df_map.items():
    print(label)
    display(df)